<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загружаем-координаты-и-готовим-данные-для-визуализации" data-toc-modified-id="Загружаем-координаты-и-готовим-данные-для-визуализации-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загружаем координаты и готовим данные для визуализации</a></span></li><li><span><a href="#Построим-геоплот" data-toc-modified-id="Построим-геоплот-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Построим геоплот</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Визуализация показателя больниц на географической карте

In [328]:
import pandas as pd
import numpy as np
import folium
import geojson
import requests
from folium.plugins import MarkerCluster
from geojson import Feature, Polygon, FeatureCollection
from seaborn import color_palette

Данные о координатах получены из открытого источника НСИ по данным ФРМО
https://nsi.rosminzdrav.ru/dictionaries/1.2.643.5.1.13.13.99.2.114/passport/2.1441

Цель работы - визуализировать на геоплоте значения показателей для больниц. Показатель - доля дистанционных записей от общего числа посещений. Количество дистанционных записей получается из баз медицинских информационных систем, количество посещений из базы территориального фонда обязательного медицинского страхования(ТФОМС). 

Мы будем использовать не реальные данные (NDA), а сгенерированные

## Загружаем координаты и готовим данные для визуализации

Из дурацкой базы с непонятной кодировкой вытаскиваем главное - OID(уникальный номер больницы) и ее координаты

In [329]:
frmo_downloads = pd.read_excel(
    'filtred.xlsx', sheet_name='Лист1', header=1).iloc[:, [32, 19, 20]]

In [330]:
frmo_downloads

,oid,latitude,longtitude
0,1.2.643.5.1.13.13.12.2.66.6930,56.885611,60.571755
1,1.2.643.5.1.13.13.12.2.66.6890,56.799350,60.463149
2,1.2.643.5.1.13.13.12.2.66.6805,56.802021,60.553978
3,1.2.643.5.1.13.13.12.2.66.6912,56.851810,60.554885
4,1.2.643.5.1.13.13.12.2.66.75616,56.920786,62.123155
...,...,...,...
162,1.2.643.5.1.13.13.12.2.66.6862,56.978217,60.565279
163,1.2.643.5.1.13.13.12.2.66.6809,56.899183,60.597726
164,1.2.643.5.1.13.13.12.2.66.6874,57.007034,61.451566
165,1.2.643.5.1.13.13.12.2.66.6901,58.049645,63.696833


Загружаем перечень OID больниц(он сделан мной для того, чтобы соединять таблицы с разными названиями больниц) и получаем из него названия больниц с координатами

In [331]:
list_oids = pd.read_excel('list_oids.xlsx', index_col=0)

In [332]:
frmo_coord = (frmo_downloads
              .merge(list_oids.drop_duplicates('oid'), how='left', on='oid')
              .dropna()
              .loc[:, ['oid', 'hospital', 'latitude', 'longtitude']])

In [333]:
frmo_coord.columns = ['oid', 'hospital', 'latitude', 'longitude']

In [334]:
frmo_coord

,oid,hospital,latitude,longitude
0,1.2.643.5.1.13.13.12.2.66.6930,"МБУ ""ЕКДЦ"" не актуально",56.885611,60.571755
1,1.2.643.5.1.13.13.12.2.66.6890,Государственное автономное учреждение здравоох...,56.799350,60.463149
2,1.2.643.5.1.13.13.12.2.66.6805,МБУ ДГБ №5,56.802021,60.553978
3,1.2.643.5.1.13.13.12.2.66.6912,"ГАУЗ СО ""Областной наркологический диспансер""",56.851810,60.554885
4,1.2.643.5.1.13.13.12.2.66.75616,ГАУЗ СО ОСЦМР Санаторий Обуховский филиал Сана...,56.920786,62.123155
...,...,...,...,...
162,1.2.643.5.1.13.13.12.2.66.6862,"ГАУЗ СО ""ВЕРХНЕПЫШМИНСКАЯ ЦГБ ИМ. П.Д. БОРОДИНА""",56.978217,60.565279
163,1.2.643.5.1.13.13.12.2.66.6809,ГАУЗ СО ДГБ 15,56.899183,60.597726
164,1.2.643.5.1.13.13.12.2.66.6874,"ГАУЗ СО ""СП Г. АСБЕСТ""",57.007034,61.451566
165,1.2.643.5.1.13.13.12.2.66.6901,"ГАУЗ СО ""Туринская ЦРБ им. О.Д. Зубова""",58.049645,63.696833


Загружаем базу с показателями(правда показатели из нее удалены ибо NDA)))

In [397]:
feature_1 = pd.read_excel('empty_hospitals.xlsx')
feature_1

,hospital,oid
0,"ГБУЗ СО ""ОКМЦ ФИЗ""",1.2.643.5.1.13.13.12.2.66.6881
1,"ГАУЗ СО ""СП №12""",1.2.643.5.1.13.13.12.2.66.6802
2,ГАУЗ СО ЦГКБ 3,1.2.643.5.1.13.13.12.2.66.6827
3,"ГАУЗ СО ""Верхнепышминская СП""",1.2.643.5.1.13.13.12.2.66.6873
4,"ГАУЗ СО ""Красноуральская СП""",1.2.643.5.1.13.13.12.2.66.6856
...,...,...
116,"ГАУЗ СО ""ОНБ""",1.2.643.5.1.13.13.12.2.66.10224
117,ГАУЗ СО ПТД № 3,1.2.643.5.1.13.13.12.2.66.6778
118,ГАУЗ СО Психиатрическая больница №3,1.2.643.5.1.13.13.12.2.66.6808
119,"ГАУЗ СО ""ПБ № 7"" Нижний Тагил",1.2.643.5.1.13.13.12.2.66.6826


Сгенерируем данные

distant - количество дистанционных записей к врачу, amount_visits - количество посещений по данным ТФОМС. 

In [398]:
feature_1['distant'] = -1

In [399]:
feature_1['amount_visits'] = np.nan

In [400]:
def fill_values(row):
    """
    Функция для случайной генерации данных о показателях больниц.

    Input:
    Строка базы данных, полученная например при момощи функции apply

    Output:
    Строка базы данных с сгенерированными показателями
    """
    while row.distant < 0:
        row.distant = int(np.random.normal(5000, 12000))
    _i = -1
    while _i < 0:
        _i = np.random.normal(0, row.distant*0.4)
        row.amount_visits = int(row.distant + _i)
    if np.random.random(1)[0] <= 0.05:
        _tmp = np.random.random(1)[0]
        if _tmp <= 0.33:
            row.distant = np.nan
        elif 0.66 > _tmp > 0.33:
            row.amount_visits = np.nan
        else:
            row.distant, row.amount_visits = np.nan, np.nan
    return row

In [401]:
feature_1 = feature_1.apply(lambda x: fill_values(x), axis=1)

In [402]:
feature_1

,hospital,oid,distant,amount_visits
0,"ГБУЗ СО ""ОКМЦ ФИЗ""",1.2.643.5.1.13.13.12.2.66.6881,NaN,NaN
1,"ГАУЗ СО ""СП №12""",1.2.643.5.1.13.13.12.2.66.6802,2993.0,3998.0
2,ГАУЗ СО ЦГКБ 3,1.2.643.5.1.13.13.12.2.66.6827,7232.0,8287.0
3,"ГАУЗ СО ""Верхнепышминская СП""",1.2.643.5.1.13.13.12.2.66.6873,10198.0,10355.0
4,"ГАУЗ СО ""Красноуральская СП""",1.2.643.5.1.13.13.12.2.66.6856,11946.0,17808.0
...,...,...,...,...
116,"ГАУЗ СО ""ОНБ""",1.2.643.5.1.13.13.12.2.66.10224,1356.0,1689.0
117,ГАУЗ СО ПТД № 3,1.2.643.5.1.13.13.12.2.66.6778,16436.0,19859.0
118,ГАУЗ СО Психиатрическая больница №3,1.2.643.5.1.13.13.12.2.66.6808,530.0,579.0
119,"ГАУЗ СО ""ПБ № 7"" Нижний Тагил",1.2.643.5.1.13.13.12.2.66.6826,12576.0,15718.0


In [403]:
feature_1.distant.median()

10869.0

In [404]:
feature_1.amount_visits.median()

13591.5

In [405]:
feature_1.isna().sum()

hospital         0
oid              0
distant          4
amount_visits    3
dtype: int64

Пропуски оставим как есть - моделирование отсутствующих данных.
Посчитаем показатель.

In [406]:
feature_1['feature'] = feature_1.distant / feature_1.amount_visits

Проверим для какого количества больниц возможно расчитать показатель(не о всех больницах есть данные в МИС или ТФОМС)

In [407]:
count_f = 0
count_t = 0
for j in frmo_coord.oid.unique():
    if j in feature_1.oid.unique():
        count_t = count_t + 1
    else:
        count_f = count_f + 1
print(f"Количество больниц с координатами: {len(frmo_coord.oid.unique())}")
print(f"Количество больниц для которых есть данные: {count_t}")
print(f"Количество больниц для которых данных нет: {count_f}")

Количество больниц с координатами: 166
Количество больниц для которых есть данные: 121
Количество больниц для которых данных нет: 45


## Построим геоплот

Зададим цвета для будующих маркеров. Возьмем palette из seaborn за основу

In [408]:
colors = color_palette("blend:#ff0000,#008000", n_colors=101, ).as_hex()

Определим функцию для создания маркеров

In [409]:
feature_1[feature_1.oid == '1.2.643.5.1.13.13.12.2.66.6873'].feature.isna()

3    False
Name: feature, dtype: bool

In [410]:
def circles_creation_frmo(row):
    """
    Функция для определения цвета маркера больницы в зависимости от значения показателя

    Input:
    Строка базы данных

    Output:
    Пустой вывод. Глобальная переменная markers - объект типа FeatureCollection,
    в который будут добавляться маркеры с определенным цветом

    """
    global markers
    # Проверяем есть ли для больницы значение показателя
    if row.oid in feature_1.oid.unique():
        # обрабатываем пропуски в таблице в зависимости от колонки в подсказку будет выведен свой текст
        # Цвет маркеров с пропусками - черный
        if (feature_1.loc[feature_1.oid == row.oid, 'distant'].isna().values[0] and
                feature_1[feature_1.oid == row.oid].amount_visits.isna().values[0]):

            feature = 'Нет данных в МИС и ТФОМС'
            visits = 'Нет данных'
            color = 'black'

        elif feature_1.loc[feature_1.oid == row.oid, 'distant'].isna().values[0]:
            feature = 'Нет данных в МИС'
            visits = feature_1[feature_1.oid ==
                               row.oid].amount_visits.values[0]
            color = 'black'

        elif feature_1[feature_1.oid == row.oid].amount_visits.isna().values[0]:
            feature = 'Нет данных в ТФОМС'
            visits = 'Нет данных'
            color = 'black'

        else:
            if feature_1[feature_1.oid == row.oid].feature.isna().values[0]:
                feature = 'Нет посещений в больнице'
                visits = feature_1[feature_1.oid ==
                                   row.oid].amount_visits.values[0]
                color = 'black'
            else:
                feature = '{:.2%}'.format(
                    feature_1.loc[feature_1.oid == row.oid, 'feature'].values[0])
                visits = feature_1[feature_1.oid ==
                                   row.oid].amount_visits.values[0]
                color = colors[round(
                    feature_1[feature_1.oid == row.oid].feature.values[0] * 100)]
    # Если больница отсутствует в расчете показателя - ее цвет серый
    else:
        color = 'grey'
        feature = 'Нет данных'
        visits = 'Нет данных'
    marker = folium.Circle(
        [row.latitude, row.longitude], tooltip=row.hospital,
        popup=f"Количество посещений: {visits}, Значение показателя: {feature}", radius=300, color=color, fill_color=color
    )
    markers.add_child(marker)
    return

Создаем карту

In [411]:
m = folium.Map([58.8519, 65.6122], zoom_start=6, min_zoom=6)

Задаем границы округов разного цвета. Для начала загрузим координаты округов

In [412]:
with open('final_district.geojson', 'r') as f:
    geo_so = geojson.load(f)

Границы округов получены вручную из OSM

In [413]:
with open('export.geojson', 'r', encoding='utf-8') as f:
    districts = geojson.load(f)

Отфильтруем нужные нам границы округов

In [414]:
filtred_districts = []

In [415]:
for j in range(len(districts['features'])):
    if districts['features'][j]['geometry']['type'] == 'Polygon':
        districts['features'][j]['properties']['name'] = districts['features'][j]['properties']['wikipedia'].replace(
            'ru:', '')
        filtred_districts.append(districts[j])

In [416]:
districts['features'] = filtred_districts

In [417]:
def style_function(feature):
    return {
        "color": "#FF0000"
    }

In [418]:
borders_1 = folium.GeoJson(districts, name="Управленческие округа",
                           style_function=style_function, marker=False)
borders_1.add_child(folium.features.GeoJsonTooltip(['name'], labels=False))
borders_1.add_to(m)

In [419]:
borders_2 = folium.GeoJson(geo_so, name="Городские округа")
borders_2.add_child(folium.features.GeoJsonTooltip(['name'], labels=False))
borders_2.add_to(m)

In [420]:
markers = folium.FeatureGroup(name='Медицинские организации')
frmo_coord.dropna().apply(lambda x: circles_creation_frmo(x), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
162    None
163    None
164    None
165    None
166    None
Length: 166, dtype: object

In [421]:
markers.add_to(m)

In [422]:
s = folium.LayerControl()
s.add_to(m)

In [423]:
m.keep_in_front(markers)

In [424]:
m

In [425]:
m.save('map_feature_1.html')

## Выводы

Серым были отмечены медицинские организации, которые не участвуют в расчете показателя

Черным - те по которым нет той или иной информации

Цветом от красного до зеленого уровень показателя в порядке возрастания

* В данной итерации случайных данных можно заметить, что наибольшие проблемы с показателем в Дегтярске и Сысерти, возможно у них плохо составлено расписание для госуслуг
* В Демидовской ГБ, СОКПБ, Верхнесалдинской ГБ тоже плохие показатели, хотя в больницах их округов показатели не так уж и плохи. Так же необходимо оценить правильность составления расписаний врачей и работы колл-центра
* Необходимо выяснить причину отсутвия данных в Верхнейвинской ГП, ГБ № 4 в Тагиле, ОКМЦ ФИЗ. Возможные причины: техническая ошибка, непредоставление данных в информационные системы в срок, некоторые больницы не работают по системе обязательного медицинского страхования (такие как психиатрические больницы), поэтому о них нет информации в ТФОМС(это в реальности, а не на сгенерированных данных)